In [36]:
import numpy as np
import pandas as pd


In [37]:
books = pd.read_csv(
    "dataset_books.csv",
    encoding="latin-1",
    on_bad_lines="skip",
    # sep=";",
    # delimiter=";"
    # error_bad_lines=False,
)


books.shape


books.head(2)


/tmp/ipykernel_76406/810150664.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [38]:
books.columns

books.columns = books.columns.str.strip()


books = books[
    [
        "ISBN",
        "Book-Title",
        "Book-Author",
        "Year-Of-Publication",
        "Publisher",
    ]
]


books.rename(
    columns={
        "ISBN": "ISBN",
        "Book-Title": "title",
        "Book-Author": "author",
        "Year-Of-Publication": "yop",
        "Publisher": "publisher",
    },
    inplace=True,
)

books.head()


,ISBN,title,author,yop,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [39]:
print(books.columns.tolist())
for col in books.columns:
    print(repr(col))


['ISBN', 'title', 'author', 'yop', 'publisher']
'ISBN'
'title'
'author'
'yop'
'publisher'


In [40]:
users = pd.read_csv(
    "dataset_users.csv",
    on_bad_lines="skip",
    encoding="latin-1",
)

users.head(2)
# users.columns


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [41]:
users.rename(
    columns={
        "User-ID": "user_id",
        "Location": "location",
        "Age": "age",
    },
    inplace=True,
)

users.head(2)


,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [42]:
ratings = pd.read_csv(
    "dataset_ratings.csv",
    on_bad_lines="skip",
    encoding="latin-1",
)


ratings.head(2)


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [43]:
ratings.columns


ratings.rename(
    columns={
        "User-ID": "user_id",
        "ISBN": "ISBN",
        "Book-Rating": "rating",
    },
    inplace=True,
)


In [44]:
print(books.shape)
print(users.shape)
print(ratings.shape)


(271360, 5)
(278858, 3)
(1149780, 3)


In [45]:
ratings.head()

uid_high_ratings = ratings["user_id"].value_counts() > 200
# ratings[ratings["user_id"].value_counts() > 200]

# ratings[ratings["user_id"]== 11676].head()


In [46]:
y = uid_high_ratings[uid_high_ratings].index
y


Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       116122,  44296,  28634,  59727,  73681, 274808, 188951,   9856, 155916,
       268622],
      dtype='int64', name='user_id', length=899)

In [47]:
ratings = ratings[ratings["user_id"].isin(y)]
ratings


,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0
...,...,...,...
1147612,275970,3829021860,0
1147613,275970,4770019572,0
1147614,275970,896086097,0
1147615,275970,9626340762,8


In [48]:
# ratings.head()

ratings_with_books = ratings.merge(books, on="ISBN")
ratings_with_books.shape

ratings_with_books.groupby("title")


In [49]:
number_of_ratings = (
    ratings_with_books.groupby("title")["rating"]
    .count()
    .reset_index()
    # .rename(
    #     columns={"ratings": "number_of_ratings"},
    #     # inplace=True,
    # )
)


In [50]:
number_of_ratings.rename(
    columns={"ratings": "number_of_ratings"},
    inplace=True,
)

print(number_of_ratings)


                                                    title  rating
0        A Light in the Storm: The Civil War Diary of ...       2
1                                   Always Have Popsicles       1
2                    Apple Magic (The Collector's series)       1
3        Beyond IBM: Leadership Marketing and Finance ...       1
4        Clifford Visita El Hospital (Clifford El Gran...       1
...                                                   ...     ...
160264  Ã?Ã?ber die Pflicht zum Ungehorsam gegen den...       3
160265                                    Ã?Ã?lpiraten.       1
160266                   Ã?Ã?rger mit Produkt X. Roman.       1
160267                            Ã?Ã?stlich der Berge.       1
160268                                Ã?Ã?thique en toc       1

[160269 rows x 2 columns]


In [51]:
final_rating = ratings_with_books.merge(number_of_ratings, on="title")
final_rating.head()


,user_id,ISBN,rating_x,title,author,yop,publisher,rating_y
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,1
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,1
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,13


In [52]:
final_rating = final_rating[final_rating["rating_y"] >= 50]

final_rating.head()


,user_id,ISBN,rating_x,title,author,yop,publisher,rating_y
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,133
15,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,108
18,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,108
24,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,79


In [53]:
final_rating.drop_duplicates(["user_id", "title"], inplace=True)

book_pivot = final_rating.pivot_table(
    columns="user_id",
    index="title",
    values="rating_y",
)


book_pivot.head()


user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,76.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,124.0,NaN,NaN,NaN,NaN,124.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,54.0,NaN,NaN,NaN,NaN


In [54]:
# import seaborn as sns
#
#
# sns.scatterplot(book_pivot)


book_pivot.fillna(0, inplace=True)
book_pivot.head()


user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,76.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,124.0,0.0,0.0,0.0,0.0,124.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,54.0,0.0,0.0,0.0,0.0


In [55]:
from scipy.sparse import csr_matrix

book_sparse = csr_matrix(book_pivot)


book_pivot.to_csv("book_pivot.csv")
type(book_sparse)


scipy.sparse._csr.csr_matrix

In [56]:
from sklearn.neighbors import NearestNeighbors
import joblib

model = NearestNeighbors(algorithm="brute")
model.fit(book_sparse)


np.where(book_pivot.index == 237)

# save model
joblib.dump(model, "knn_model.pkl")


['knn_model.pkl']

In [57]:
book_index = 237

distances, suggestions = model.kneighbors(
    book_pivot.iloc[book_index, :].values.reshape(1, -1)
)


# print(book_pivot)
print(suggestions)
print(distances)


[[237 240 238 239 241]]
[[   0.         1753.17340842 1822.42201479 2082.78227379 2190.89776119]]


In [58]:
book_pivot.index[book_index]  # suggestions


'Harry Potter and the Chamber of Secrets (Book 2)'

In [59]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])


Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Order of the Phoenix (Book 5)',
       'Harry Potter and the Sorcerer's Stone (Book 1)'],
      dtype='object', name='title')


In [60]:
np.where(book_pivot.index == "Animal Farm")[0][0]


np.int64(54)

In [61]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]

    distances, suggestions = model.kneighbors(
        book_pivot.iloc[book_id, :].values.reshape(1, -1)
    )

    # for i in range(len(suggestions)):
    #     if i != 0:
    #         print(f"The suggestions for {book_name}({book_id}) are :\n")
    #         print(book_pivot.index[suggestions[i]])
    #     else:
    #         print("None Found")

    print(f"The suggestions for {book_name}({book_id}) are :\n")

    for idx in suggestions[0][1:]:  # skip index 0 (the book itself)
        print(book_pivot.index[idx])


# recommend_book("Animal Farm")


In [62]:
recommend_book("Harry Potter and the Chamber of Secrets (Book 2)")


The suggestions for Harry Potter and the Chamber of Secrets (Book 2)(237) are :

Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Order of the Phoenix (Book 5)
Harry Potter and the Sorcerer's Stone (Book 1)
